In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
df = pd.read_parquet('../data/electronics_sample_2M.parquet')

In [3]:
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   reviewerID      object 
 1   asin            object 
 2   overall         float32
 3   unixReviewTime  int64  
 4   title           object 
 5   brand           object 
 6   categories      object 
dtypes: float32(1), int64(1), object(5)
memory usage: 99.2+ MB
None


,reviewerID,asin,overall,unixReviewTime,title,brand,categories
0,AHAGVEUWD2IRJFPCJLOAYSJ7XNLQ,B008YQAG5Q,5.0,1393181267000,HP BD-R DL 6X 50GB Double Layer 10 Pack in Spi...,HP,"[Electronics, Computers & Accessories, Compute..."
1,AHYKM5KHAJNN3KQ2EXO25LCCKDFA,B0711V1WXC,2.0,1524659491257,Fintie Slim Case for Amazon Fire 7 Tablet (Pre...,Fintie,"[Electronics, Computers & Accessories, Tablet ..."
2,AHZC5B3S7CRPM5S476EIQJKWUS6Q,B00DOHVUIM,5.0,1414421305000,Kuzy Compatible with MacBook Keyboard Cover fo...,Kuzy,"[Electronics, Computers & Accessories, Compute..."
3,AGRTA2KJTZWKSFVLMSK2K7RGAKMQ,B000BNY64C,4.0,1376084786000,STK BP-511a BP-511 2 Pack Battery for Canon Re...,SterlingTek,"[Electronics, Camera & Photo, Accessories, Bat..."
4,AEYUTNEKMD56ZZQ3ZC7XOJDHGRLQ,B0BT9MK3XK,5.0,1613463323771,"Nulaxy Tablet Stand, Fully Adjustable Foldable...",Nulaxy,"[Electronics, Computers & Accessories, Tablet ..."


## Target Variable (binary)

In [4]:
# 1 if 'overall' is >= 4.0, and 0 otherwise.
df['positive'] = (df['overall'] >= 4.0).astype(int)


print("\nTarget variable distribution:")
print(df['positive'].value_counts(normalize=True))


Target variable distribution:
positive
1    0.800894
0    0.199106
Name: proportion, dtype: float64


## Light Feature Cleaning

In [5]:
missing_title = df['title'].isnull().sum()
missing_brand = df['brand'].isnull().sum()
null_categories = df['categories'].isnull().sum()

print(missing_title)
print(missing_brand)
print(null_categories)

0
1645
0


In [6]:
# Fill nulls for our text/categorical features
# print("\nChecking and filling null values...")
# df['brand'] = df['brand'].fillna('Unknown')

# print("Feature cleaning complete.")

In [7]:
# missing_brand = df['brand'].isnull().sum()
# print(missing_brand)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   reviewerID      object 
 1   asin            object 
 2   overall         float32
 3   unixReviewTime  int64  
 4   title           object 
 5   brand           object 
 6   categories      object 
 7   positive        int64  
dtypes: float32(1), int64(2), object(5)
memory usage: 114.4+ MB


## Time-Based Split

In [9]:
# 1. Sort the entire DataFrame by time
print(f"Sorting {len(df)} rows by 'unixReviewTime'...")
df_sorted = df.sort_values(by='unixReviewTime')
print("Sorting complete.")

# 2. Calculate split indices
total_rows = len(df_sorted)
train_end = int(total_rows * 0.8)  # 80% for training
val_end = int(total_rows * 0.9)    # 10% for validation (from 80% to 90%)
# The last 10% is automatically for testing

# 3. Create the splits using iloc (index-based slicing)
print("Splitting data into train, validation, and test sets...")
train_df = df_sorted.iloc[:train_end]
val_df = df_sorted.iloc[train_end:val_end]
test_df = df_sorted.iloc[val_end:]

# 4. Verify the splits
print("\nSplit complete. Data shapes:")
print(f"Train set:      {train_df.shape}")
print(f"Validation set: {val_df.shape}")
print(f"Test set:       {test_df.shape}")


Sorting 2000000 rows by 'unixReviewTime'...
Sorting complete.
Splitting data into train, validation, and test sets...

Split complete. Data shapes:
Train set:      (1600000, 8)
Validation set: (200000, 8)
Test set:       (200000, 8)


In [10]:
# Check the time boundaries
print("\nTime boundaries:")
print(f"Training data ends at:   {pd.to_datetime(train_df['unixReviewTime'].max(), unit='ms')}")
print(f"Validation data starts at: {pd.to_datetime(val_df['unixReviewTime'].min(), unit='ms')}")
print(f"Test data starts at:     {pd.to_datetime(test_df['unixReviewTime'].min(), unit='ms')}")
print(f"Test data ends at:     {pd.to_datetime(test_df['unixReviewTime'].max(), unit='ms')}")


Time boundaries:
Training data ends at:   2021-03-03 04:11:50.559000
Validation data starts at: 2021-03-03 04:12:34.086000
Test data starts at:     2022-04-04 15:16:48.134000
Test data ends at:     2023-09-10 22:15:13.085000


## Saving train, val and test

In [11]:
print("Saving data splits to Parquet...")

# Save each DataFrame to its own file
train_df.to_parquet('../data/train_df.parquet', index=False)
val_df.to_parquet('../data/val_df.parquet', index=False)
test_df.to_parquet('../data/test_df.parquet', index=False)

print("Train, validation, and test sets saved!")

Saving data splits to Parquet...
Train, validation, and test sets saved!


## Integer Mapping for categorial features

* convert our main categorical columns (reviewerID, asin, brand annd categories) into integer IDs.

The correct way is to:

Fit: Create the mapping (e.g., {'Sony': 1, 'Apple': 2}) using only train_df.

ID 0: "This is a specific brand/item, but I haven't learned its personality yet." Value is valid but unseen (Cold Start)
ID 1: "This item has NO brand info at all." means Value is NaN / None

Transform: Use that same mapping on train_df, val_df, and test_df.

In [12]:
print("Starting categorical feature engineering (Robust Method)...")

# --- 1. Helper to Build Mappings (Ignoring NaNs) ---
def create_and_save_mapping(dataframe, column_name, output_filename):
    """
    Creates a vocabulary from a DataFrame column.
    - Ignores NaN values when building the list.
    - Reserves ID 0 for <UNKNOWN> (unseen values).
    - Reserves ID 1 for <MISSING> (NaN values).
    - Starts real IDs at 2.
    """
    print(f"Creating mapping for: {column_name}")
    
    # Get unique values, BUT explicitly drop NaNs first
    unique_values = dataframe[column_name].dropna().unique()
    
    # Create the mapping starting at ID 2
    mapping = {val: i+2 for i, val in enumerate(unique_values)}
    
    # Add our special tokens
    mapping['<UNKNOWN>'] = 0 
    mapping['<MISSING>'] = 1 
    
    # Save to JSON
    with open(output_filename, 'w') as f:
        # We need to convert numpy types to standard python types for JSON
        # (e.g. numpy.int64 -> int)
        def convert(o):
            if isinstance(o, np.integer): return int(o)
            raise TypeError
        json.dump(mapping, f, default=convert)
        
    print(f"Saved mapping with {len(mapping)} entries to {output_filename}")
    return mapping

# --- 2. Create mappings (from TRAIN only) ---
user_mapping = create_and_save_mapping(train_df, 'reviewerID', '../data/user_mapping.json')
item_mapping = create_and_save_mapping(train_df, 'asin', '../data/item_mapping.json')
brand_mapping = create_and_save_mapping(train_df, 'brand', '../data/brand_mapping.json')


# --- 3. Helper to Apply Mappings (Handling NaNs) ---
def apply_mapping_robust(dataframe, column_name, mapping):
    """
    Applies mapping with explicit logic:
    - If NaN -> Return 1 (<MISSING>)
    - If in Map -> Return ID
    - If Not in Map -> Return 0 (<UNKNOWN>)
    """
    print(f"Applying mapping to: {column_name}")
    
    def get_id(val):
        # Case 1: Value is NaN / None
        if pd.isna(val) or val == "":
            return mapping['<MISSING>'] # ID 1
        
        # Case 2: Value is in our vocabulary
        if val in mapping:
            return mapping[val]
        
        # Case 3: Value is valid but unseen (Cold Start)
        return mapping['<UNKNOWN>'] # ID 0

    return dataframe[column_name].apply(get_id)



Starting categorical feature engineering (Robust Method)...
Creating mapping for: reviewerID
Saved mapping with 915325 entries to ../data/user_mapping.json
Creating mapping for: asin
Saved mapping with 235824 entries to ../data/item_mapping.json
Creating mapping for: brand
Saved mapping with 31325 entries to ../data/brand_mapping.json


In [14]:
# --- FIX for SettingWithCopyWarning ---
print("Creating independent copies of the data splits...")

# The .copy() method creates a brand new object in memory
train_df = train_df.copy()
val_df = val_df.copy()
test_df = test_df.copy()

Creating independent copies of the data splits...


In [15]:

# --- 4. Apply to all splits ---
print("\nApplying mappings to train, val, and test sets...")

datasets = [train_df, val_df, test_df]
# columns: (new_col_name, old_col_name, mapping_dict)
mapping_configs = [
    ('user_id', 'reviewerID', user_mapping),
    ('item_id', 'asin', item_mapping),
    ('brand_id', 'brand', brand_mapping)
]

for df in datasets:
    for new_col, old_col, mapping in mapping_configs:
        df[new_col] = apply_mapping_robust(df, old_col, mapping)

# --- 5. Verify ---
print("\nMapping complete. Checking 'brand_id' for missing values handling:")
# Let's check a row where brand might be missing if you have one, 
# or just print the first few rows.
print(train_df[['brand', 'brand_id']].head(10))

# Check if we have any ID=1 (Missing) in the brand_id column
missing_count = (train_df['brand_id'] == 1).sum()
print(f"\nRows with Missing Brand (ID=1) in Train: {missing_count}")


Applying mappings to train, val, and test sets...
Applying mapping to: reviewerID
Applying mapping to: asin
Applying mapping to: brand
Applying mapping to: reviewerID
Applying mapping to: asin
Applying mapping to: brand
Applying mapping to: reviewerID
Applying mapping to: asin
Applying mapping to: brand

Mapping complete. Checking 'brand_id' for missing values handling:
             brand  brand_id
1190044         HP         2
1236487  Replay Tv         3
63617      Olympus         4
779636     PHILIPS         5
1673857    Olympus         4
506328   Microsoft         6
1125448    PHILIPS         5
862728   Replay Tv         3
1402558    Toshiba         7
1841846         HP         2

Rows with Missing Brand (ID=1) in Train: 1527


## Task 6: Process Categories
Extracts the single best category string from the list.

Maps that string to an integer ID (using the same robust logic).

In [26]:
import numpy as np
import ast

print("Starting 'categories' processing (NumPy Support Added)...")

def get_most_specific_category(val):
    # --- CASE 1: Handle NumPy Arrays (The likely culprit!) ---
    if isinstance(val, np.ndarray):
        val = val.tolist()
    
    # --- CASE 2: Handle Strings that look like lists ---
    elif isinstance(val, str) and val.startswith('['):
        try:
            val = ast.literal_eval(val)
        except:
            return 'Unknown'
            
    # --- CASE 3: If it's still not a list, fail ---
    if not isinstance(val, list):
        return 'Unknown'
    
    # --- Now process the List ---
    if len(val) == 0:
        return 'Unknown'
        
    try:
        # Check for nested list [['Elec', 'Comp']]
        # (Parquet sometimes flattens this, so we check carefully)
        first_item = val[0]
        if isinstance(first_item, list) or isinstance(first_item, np.ndarray):
             # It's nested, grab the last sub-list
             last_sublist = val[-1]
             if len(last_sublist) > 0:
                 return last_sublist[-1]
             else:
                 return 'Unknown'
        else:
            # It's a simple flat list ['Elec', 'Comp']
            return val[-1]
    except Exception as e:
        return 'Unknown'

# --- Apply extraction ---
print("Extracting 'main_category' column...")
for df in [train_df, val_df, test_df]:
    df['main_category'] = df['categories'].apply(get_most_specific_category)

print("Extraction example:")
print(train_df[['categories', 'main_category']].head())

# --- Re-run Mapping (Standard Steps) ---
category_mapping = create_and_save_mapping(train_df, 'main_category', '../data/category_mapping.json')

print("\nApplying 'main_category_id' mapping...")
for df in [train_df, val_df, test_df]:
    df['main_category_id'] = apply_mapping_robust(df, 'main_category', category_mapping)

print("Category processing complete!")
print(train_df[['main_category', 'main_category_id']].head())

Starting 'categories' processing (NumPy Support Added)...
Extracting 'main_category' column...
Extraction example:
                                                categories  \
1190044  [Electronics, Computers & Accessories, Compute...   
1236487  [Electronics, Television & Video, Streaming Me...   
63617    [Electronics, Camera & Photo, Film Photography...   
779636   [Electronics, Television & Video, Streaming Me...   
1673857  [Electronics, Camera & Photo, Digital Cameras,...   

                         main_category  
1190044                        Webcams  
1236487        Streaming Media Players  
63617       Point & Shoot Film Cameras  
779636         Streaming Media Players  
1673857  Point & Shoot Digital Cameras  
Creating mapping for: main_category
Saved mapping with 1001 entries to ../data/category_mapping.json

Applying 'main_category_id' mapping...
Applying mapping to: main_category
Applying mapping to: main_category
Applying mapping to: main_category
Category processing 

In [25]:
print(train_df['categories'].head(2))
# categories column is not a Python list, and it is not a string. It is a NumPy Array (numpy.ndarray).

1190044    [Electronics, Computers & Accessories, Compute...
1236487    [Electronics, Television & Video, Streaming Me...
Name: categories, dtype: object


In [27]:
train_df.head()

,reviewerID,asin,overall,unixReviewTime,title,brand,categories,positive,user_id,item_id,brand_id,main_category,main_category_id
1190044,AENJWAP4JGEFZGDCTSX72UQ6M7IQ,B00000JDHV,4.0,940168313000,3Com 00371800 HomeConnect PC Digital Camera,HP,"[Electronics, Computers & Accessories, Compute...",1,2,2,2,Webcams,2
1236487,AHILMS23CHY27DIIEOPI3EHMRXAQ,B00002ST80,2.0,941867224000,ReplayTV 2020 Digital Video Recorder,Replay Tv,"[Electronics, Television & Video, Streaming Me...",0,3,3,3,Streaming Media Players,3
63617,AFZXSUCRYI2REEOY6WDM32GNAGWA,B000021YU8,5.0,942565450000,Olympus Stylus Epic QD CG Date 35mm Camera,Olympus,"[Electronics, Camera & Photo, Film Photography...",1,4,4,4,Point & Shoot Film Cameras,4
779636,AFZ2RQH65E3ZYPP6UWHHWBZPFEYQ,B00002SWUE,5.0,942986043000,Philips HDR112 Tivo Digital Video Recorder,PHILIPS,"[Electronics, Television & Video, Streaming Me...",1,5,5,5,Streaming Media Players,3
1673857,AHTQFIGJM4KTAA2S6OY6JN6LF2LA,B00000JFIF,5.0,943652529000,Olympus D-340R 1.2MP Digital Camera,Olympus,"[Electronics, Camera & Photo, Digital Cameras,...",1,6,6,4,Point & Shoot Digital Cameras,5


In [28]:
print("Saving categorical processed data...")

# We'll call these '_categorical' to distinguish them from the final vectors
train_df.to_parquet('../data/train_categorical.parquet', index=False)
val_df.to_parquet('../data/val_categorical.parquet', index=False)
test_df.to_parquet('../data/test_categorical.parquet', index=False)

print("This consists of (Target variable + Categorical (user, item, brand, main_category) IDs)")

Saving categorical processed data...
This consists of (Target variable + Categorical (user, item, brand, main_category) IDs)
